<a href="https://colab.research.google.com/github/Satish-970/ATS_Checker_Job-Description/blob/main/ATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ATS Resume Matcher: Your Guide to a Better Resume

This tool helps you evaluate how well your resume matches a job description, providing an **ATS score** and suggestions for improvement. It's designed to help you pass through Applicant Tracking Systems (ATS) and get your resume seen by recruiters.

To use it, simply run all the code cells sequentially. You'll be prompted to upload your resume as a PDF and paste the job description you're applying for.

In [ ]:
# Step 1: Install and Import Libraries

# This cell installs necessary libraries and downloads the spaCy model.
!pip install spacy PyPDF2 scikit-learn
!python -m spacy download en_core_web_sm

# Import libraries
import spacy
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from google.colab import files
from IPython.display import clear_output
import os

print("Setup complete!")

In [ ]:
# Step 2: Define Core Functions

# This cell defines all the helper functions for the ATS checker.

# Load the spaCy model
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print("Downloading spaCy model...")
    !python -m spacy download en_core_web_sm
    nlp = spacy.load('en_core_web_sm')

# Function to extract text from a PDF
def extract_resume_text(pdf_path):
    """
    Extracts text from a PDF resume file.
    """
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + ' '
    return text.strip()

# Function to extract keywords (nouns, proper nouns, and custom skills)
def extract_keywords(text):
    """
    Extracts keywords (nouns, proper nouns, and custom skills) from text.
    """
    doc = nlp(text.lower())
    keywords = set()
    
    # Enhanced list of technical skills and keywords
    tech_skills = {
        'python', 'sql', 'machine learning', 'data analysis', 'java', 'javascript',
        'react', 'angular', 'node.js', 'html', 'css', 'aws', 'docker', 'kubernetes',
        'git', 'agile', 'scrum', 'tensorflow', 'pytorch', 'pandas', 'numpy',
        'mongodb', 'postgresql', 'mysql', 'excel', 'powerbi', 'tableau',
        'communication', 'leadership', 'teamwork', 'management', 'project management'
    }
    
    for token in doc:
        # Extract nouns and proper nouns
        if token.pos_ in ['NOUN', 'PROPN'] and len(token.text) > 2:
            keywords.add(token.text)
        # Extract technical skills
        elif token.text in tech_skills:
            keywords.add(token.text)
    
    # Handle multi-word skills
    text_lower = text.lower()
    multi_word_skills = [
        'machine learning', 'data analysis', 'project management', 
        'data science', 'artificial intelligence', 'deep learning',
        'natural language processing', 'computer vision', 'big data'
    ]
    
    for skill in multi_word_skills:
        if skill in text_lower:
            keywords.add(skill)
    
    return keywords

# Function to calculate similarity score using TF-IDF and cosine similarity
def calculate_ats_score(resume_text, job_description):
    """
    Computes similarity score between resume and job description using TF-IDF and cosine similarity.
    """
    documents = [resume_text, job_description]
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
    tfidf_matrix = vectorizer.fit_transform(documents)
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return similarity * 100

# Function to provide suggestions based on missing keywords
def get_suggestions(resume_keywords, job_keywords):
    """
    Finds keywords in job description that are missing from resume and provides suggestions.
    """
    missing = job_keywords - resume_keywords
    if missing:
        missing_list = list(missing)[:10]  # Limit to top 10 suggestions
        return f"Suggestions: Consider adding these keywords to your resume: {', '.join(missing_list)}"
    return "Great match! Your resume contains all the key skills from the job description."

# Function to categorize score and provide feedback
def get_score_feedback(score):
    """
    Provides feedback based on the ATS score.
    """
    if score >= 80:
        return "Excellent! Your resume is very well-matched to this job."
    elif score >= 60:
        return "Good match! Consider minor improvements to boost your score."
    elif score >= 40:
        return "Moderate match. Review the suggestions to improve your resume."
    else:
        return "Low match. Consider significant revisions to better align with the job requirements."

# Function to visualize the ATS score
def visualize_score(score):
    """
    Plots a bar chart of the ATS score with color coding.
    """
    # Color coding based on score
    if score >= 80:
        color = 'green'
    elif score >= 60:
        color = 'orange'
    elif score >= 40:
        color = 'yellow'
    else:
        color = 'red'
    
    plt.figure(figsize=(8, 6))
    plt.bar(['ATS Score'], [score], color=color, alpha=0.7)
    plt.ylim(0, 100)
    plt.ylabel('Match Percentage (%)')
    plt.title('Resume ATS Match Score')
    
    # Add score text on the bar
    plt.text(0, score + 2, f'{score:.1f}%', ha='center', va='bottom', fontsize=14, fontweight='bold')
    
    # Add threshold lines
    plt.axhline(y=80, color='green', linestyle='--', alpha=0.5, label='Excellent (80%+)')
    plt.axhline(y=60, color='orange', linestyle='--', alpha=0.5, label='Good (60%+)')
    plt.axhline(y=40, color='yellow', linestyle='--', alpha=0.5, label='Moderate (40%+)')
    
    plt.legend()
    plt.tight_layout()
    plt.show()

print("Core functions defined.")

In [ ]:
# Step 3: Run the Interactive ATS Checker

# This function guides the user through the process of uploading files and getting results.
def run_ats_checker():
    clear_output(wait=True)
    
    print("🎯 ATS Resume Checker")
    print("=" * 50)
    print("This tool will analyze how well your resume matches a job description.")
    print("\nStep 1: Please upload your resume PDF.")
    print("A file dialog will open shortly...")
    
    uploaded_files = files.upload()
    
    if not uploaded_files:
        print("❌ No file was uploaded. Please try again.")
        return
    
    file_name = list(uploaded_files.keys())[0]
    print(f"✅ Successfully uploaded: {file_name}")
    
    try:
        resume_text = extract_resume_text(file_name)
        print(f"📄 Extracted {len(resume_text)} characters from your resume.")
    except Exception as e:
        print(f"❌ Error reading PDF: {e}")
        os.remove(file_name)
        return
    
    print("\n" + "="*50)
    print("Step 2: Please paste the complete job description below:")
    job_desc = input("\nJob Description: ")
    
    if not job_desc.strip():
        print("❌ Job description cannot be empty. Please try again.")
        os.remove(file_name)
        return
    
    print("\n🔍 Analyzing...")
    
    # Extract keywords and calculate score
    resume_keywords = extract_keywords(resume_text)
    job_keywords = extract_keywords(job_desc)
    ats_score = calculate_ats_score(resume_text, job_desc)
    
    # Display results
    print("\n" + "="*60)
    print("📊 ATS MATCH ANALYSIS RESULTS")
    print("="*60)
    print(f"\n🎯 Your ATS Match Score: {ats_score:.2f}%")
    print(f"📝 {get_score_feedback(ats_score)}")
    print(f"\n💡 {get_suggestions(resume_keywords, job_keywords)}")
    
    print(f"\n📈 Resume Keywords Found: {len(resume_keywords)}")
    print(f"📋 Job Keywords Found: {len(job_keywords)}")
    print(f"🎯 Matching Keywords: {len(resume_keywords & job_keywords)}")
    
    # Show visualization
    visualize_score(ats_score)
    
    # Clean up
    os.remove(file_name)
    print(f"\n🧹 Temporary file '{file_name}' removed.")
    print("\n" + "="*60)
    print("Analysis Complete! 🎉")
    print("="*60)

# Run the main checker function to start the process
run_ats_checker()

In [ ]:
# Optional: Test the functions with sample data

# Uncomment and run this cell to test the functions without uploading files

# sample_resume = """
# John Doe
# Software Engineer
# 
# Experience:
# - 3 years of Python development
# - Machine learning projects using TensorFlow
# - SQL database management
# - Agile development methodology
# 
# Skills:
# Python, SQL, Machine Learning, TensorFlow, Git, Agile
# """

# sample_job = """
# Software Engineer Position
# 
# Requirements:
# - Python programming
# - Machine learning experience
# - SQL database knowledge
# - React.js for frontend
# - Docker containerization
# - AWS cloud services
# """

# print("Testing with sample data:")
# resume_kw = extract_keywords(sample_resume)
# job_kw = extract_keywords(sample_job)
# score = calculate_ats_score(sample_resume, sample_job)

# print(f"Sample ATS Score: {score:.2f}%")
# print(f"Resume Keywords: {sorted(resume_kw)}")
# print(f"Job Keywords: {sorted(job_kw)}")
# print(get_suggestions(resume_kw, job_kw))
# visualize_score(score)